### 1. Setup

In [1]:
import os
import json
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Load API key from .env
load_dotenv()
if not os.getenv("GROQ_API_KEY"):
    raise ValueError("Missing GROQ_API_KEY in .env")

# Init Groq model (can swap model easily)
model = init_chat_model(
    "llama-3.3-70b-versatile",
    model_provider="groq",
    temperature=0.2  # lower temp for deterministic outputs
)


### 2. Load Module-2 JSON

In [2]:
with open("sta_output.json", "r") as f:
    data = json.load(f)

paths = data.get("paths", [])  # our optimized Module-2 output
print(f"Loaded {len(paths)} timing paths.")


Loaded 2 timing paths.


### 3. Prompt Template

In [6]:
# Few-shot example (curly braces doubled)
few_shot_examples = """
### EXAMPLE
Input:
{{
  "startpoint": "U1/Q",
  "endpoint": "U5/D",
  "slack": -0.85,
  "delay": 5.2,
  "clock": "clk_main"
}}

Output:
{{
  "root_cause": "Path delay exceeds the clock period, causing setup violation.",
  "suggestions": [
    "Insert a pipeline register to break the path.",
    "Optimize combinational logic to reduce delay.",
    "Replace slow gates with faster cells."
  ]
}}
"""

prompt_violation_debugger = PromptTemplate.from_template(
    f"""
You are a GenAI-powered timing violation debugger.
You will receive one STA path at a time in JSON format.

### Input Path JSON:
{{path_json}}

### INSTRUCTION:
1. If status = "VIOLATED":
   * Identify the root cause based on path_type and logic_chain.
   * Suggest an optimal fix (1–2 sentences).
2. If status = "MET":
   * root_cause = null
   * suggested_fix = null

### OUTPUT:
Return ONLY valid JSON with these keys:
"startpoint", "endpoint", "path_type", "status", "root_cause", "suggested_fix".

{few_shot_examples}
"""
)

# JSON parser
json_parser = JsonOutputParser()


### 4. Run LLM Analysis

In [7]:
results = []

for i, p in enumerate(paths, start=1):
    # Chain: prompt + model
    chain = prompt_violation_debugger | model
    
    # Invoke LLM
    res = chain.invoke({"path_json": json.dumps(p, indent=2)})
    
    # Parse output
    parsed_res = json_parser.parse(res.content)
    results.append(parsed_res)

print("Sample result:")
print(json.dumps(results[:1], indent=2))


Sample result:
[
  {
    "startpoint": "in1 (input port clocked by clk)",
    "endpoint": "r1 (rising edge-triggered flip-flop clocked by clk)",
    "path_type": "min",
    "status": "VIOLATED",
    "root_cause": "The minimum path delay is not meeting the required timing, likely due to insufficient delay in the logic chain or issues with the clock path.",
    "suggested_fix": "Insert a delay buffer or optimize the logic chain to increase the path delay, or adjust the clock timing to better match the minimum path requirements."
  }
]


In [9]:
print(json.dumps(results, indent=2))

[
  {
    "startpoint": "in1 (input port clocked by clk)",
    "endpoint": "r1 (rising edge-triggered flip-flop clocked by clk)",
    "path_type": "min",
    "status": "VIOLATED",
    "root_cause": "The minimum path delay is not meeting the required timing, likely due to insufficient delay in the logic chain or issues with the clock path.",
    "suggested_fix": "Insert a delay buffer or optimize the logic chain to increase the path delay, or adjust the clock timing to better match the minimum path requirements."
  },
  {
    "startpoint": "r2 (rising edge-triggered flip-flop clocked by clk)",
    "endpoint": "r3 (rising edge-triggered flip-flop clocked by clk)",
    "path_type": "max",
    "status": "MET",
    "root_cause": null,
    "suggested_fix": null
  }
]


### 5. Save final analysis

In [10]:
output = {"analysis": results}

with open("llm_analysis.json", "w") as f:
    json.dump(output, f, indent=2)

print("Saved -> llm_analysis.json")


Saved -> llm_analysis.json
